In [1]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

# Set the path to the directory containing the project
sys.path.append('D:\\Internship\\recsys\\db_handlers')
sys.path.append('D:\\Internship\\recsys\\movie_recommendation_system\\src')

In [2]:
from db_handlers.kg_rag_neo4j_db_handler import (
    create_knowledge_graph,
    reset_knowledge_graph,
    store_movie_rating,
    #store_new_movies_cast_and_crew_nodes,
    get_movie_cast_and_crew_information,
    show,
)

c:\Users\Public\anaconda3\envs\tf-gpu\Lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


[LOG] Detected local environment, with Docker: [False]


In [3]:
from movie_recommender.data.tabular_dataset_handler import TabularDatasetHandler

data_dir = 'D:\\Internship\\recsys\\data\\movielens'

# Generate data to be stored in the tables
tdh = TabularDatasetHandler(data_dir)
tdh.load_datasets()
tdh.preprocess_datasets()

# data_dir = 'D:\\Internship\\recsys\\data\\movielens_processed\\tdh_instance.pkl'

# tdh = TabularDatasetHandler.load_class_instance(filepath=data_dir)

D:\Internship\recsys\movie_recommendation_system\src\movie_recommender\data\tabular_dataset_handler.py:265 DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.

In [4]:
movies_df = tdh.get_movies_df_deepcopy()
credits_df = tdh.get_credits_df_deepcopy()

In [5]:
# Merge movies and credits DataFrames
merged_df = movies_df.merge(credits_df, on='id', how='left')
merged_df = merged_df[["id", "title", "budget", "genres", "overview", "production_companies", "production_countries", "release_date", "revenue", "runtime", "spoken_languages", "tagline", "cast", "crew"]]

# Preprocessing
merged_df['tagline'] = merged_df['tagline'].replace({np.nan: ""})
merged_df['budget'] = merged_df['budget'].replace({np.nan: -1, 0: -1})
merged_df['overview'] = merged_df['overview'].replace({np.nan: ""})
merged_df['release_date'] = merged_df['release_date'].replace({np.nan: ""})
merged_df['revenue'] = merged_df['revenue'].replace({np.nan: -1, 0: -1})
merged_df['runtime'] = merged_df['runtime'].replace({np.nan: -1, 0: -1})
merged_df['cast'] = merged_df['cast'].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else []
)
merged_df['crew'] = merged_df['crew'].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else []
)

merged_df.head(2)

,id,title,budget,genres,overview,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,cast,crew
0,862,Toy Story,30000000,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],,"[{'character': 'Woody (voice)', 'name': 'Tom H...","[{'department': 'Directing', 'job': 'Director'..."
1,8844,Jumanji,65000000,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Roll the dice and unleash the excitement!,"[{'character': 'Alan Parrish', 'name': 'Robin ...","[{'department': 'Production', 'job': 'Executiv..."


In [6]:
# row = merged_df.iloc[1]
# print(row.to_dict())

In [7]:
# rows_with_nan = merged_df[merged_df.isnull().any(axis=1)]

# # Print which value is NaN in each row
# for index, row in rows_with_nan.iterrows():
#     nan_columns = row[row.isnull()].index.tolist()
#     print(f"Row {index} has NaN in columns: {nan_columns}")
#     print(row.to_dict())

In [8]:
# Create knowledge graph
create_knowledge_graph(movies_cast_and_crew_df=merged_df)

D:\Internship\recsys\db_handlers\db_handlers\kg_rag_neo4j_db_handler.py:51 LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.

[2025-04-25T17:17:40.885+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Movie) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_2c627a06 FOR (e:Movie) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE;'
[2025-04-25T17:17:40.895+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Person) REQUIRE (e.name) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_a831e4ce FOR (e:Person) REQUIRE (e.name) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIR

Ingesting batches: 100%|██████████| 6/6 [00:29<00:00,  4.97s/it]

[LOG] Creating/Ensuring full-text indices...
[2025-04-25T17:18:11.096+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE FULLTEXT INDEX movie IF NOT EXISTS FOR (e:Movie) ON EACH [e.title]` has no effect.} {description: `FULLTEXT INDEX movie FOR (e:Movie) ON EACH [e.title]` already exists.} {position: None} for query: 'CREATE FULLTEXT INDEX movie IF NOT EXISTS FOR (m:Movie) ON EACH [m.title]'
[2025-04-25T17:18:11.101+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE FULLTEXT INDEX person IF NOT EXISTS FOR (e:Person) ON EACH [e.name]` has no effect.} {description: `FULLTEXT INDEX person FOR (e:Person) ON EACH [e.name]` already exists.} {position: None} for query: 'CREATE FULLTEXT INDEX person IF NOT 

In [9]:
# Example: Rate 'Toy Story'. Assumes 'Toy Story' resolves to a single movie.
# If 'Toy Story' were ambiguous, the function would print the options and return an error message.
# The function now expects the movie title, not the ID.
rating_response = store_movie_rating(user_id="test_user_1", movie_id="Toy Story", rating=5)
print(rating_response)

Need additional information, which of these did you mean: 
{'candidate': 'Toy Story', 'label': 'Movie', 'id': '862'}
{'candidate': 'Toy Story 3', 'label': 'Movie', 'id': '10193'}
{'candidate': 'Toy Story 2', 'label': 'Movie', 'id': '863'}
Please provide the 'id' for the movie.


In [10]:
cypher_query = "MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 1000"
show(cypher_query)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [11]:
# Initial lookup for 'Titanic' - likely ambiguous
# We call it without the entity_id parameter first.
movie_info_ambiguous = get_movie_cast_and_crew_information("Titanic", "movie")
print("--- Ambiguous Result ---")
print(movie_info_ambiguous)

--- Ambiguous Result ---
Need additional information, which of these did you mean: 
{'candidate': 'Titanica', 'label': 'Movie', 'id': '45929'}
{'candidate': 'Titanic', 'label': 'Movie', 'id': '597'}
{'candidate': 'Titanic', 'label': 'Movie', 'id': '16535'}
Please call the function again providing the 'id' for the desired entity.


In [12]:
# Follow-up lookup using a specific ID
specific_movie_id = "597"

# Call the function again, providing the entity_id
movie_info_specific = get_movie_cast_and_crew_information(entity="Titanic", type="movie", entity_id=specific_movie_id)
print(f"\n--- Specific Result for ID: {specific_movie_id} ---")
print(movie_info_specific)


--- Specific Result for ID: 597 ---
type:Movie
title: Titanic
year: 1997
IN_GENRE: Drama, Romance, Thriller
ACTED_IN: Miguel A. Lomelin, Nicholas Cascone, Tricia O'Neil, Mark Capri, James Lancaster, Brian Baines, John Slade, Mandana Marino, James Cameron, Greg Ellis, Paul Brightwell, Charlotte Chatton, Kate Winslet, Alexandrea Owens, Kevin De La Noy, Richard Fox, Vern Urich, Shay Duffin, Kathy Bates, Olivia Rosewood, Terry Forrestal, Edmond Alan Forsyth, Edward Fletcher, Alexandra Boyd, Jason Barry, Scott Hislop, Lew Palter, Reece P. Thompson III, Kris Andersson, Rocky Taylor, Eric Braeden, Richard Graham, Leonardo DiCaprio, Meghan McLeod, Bret Aaron Knower, Michael Ensign, Bernard Fox, Ioan Gruffudd, Marc Cass, Stephen Wolfe Smith, Camilla Overbye Roos, Kevyn Currie, Anne Fletcher, Martin East, Brian Walsh, Diana Morgan, Tony Kenny, Fannie Brett, Phil Parlapiano, Van Ling, Linda Kerns, Sean Howse, David Warner, Kevin Owers, Gloria Stuart, Martin Jarvis, Amy Gaipa, Billy Zane, Stan Ma

In [13]:
person_info = get_movie_cast_and_crew_information("Tom Hanks", "person")
print(person_info)

type:Person
title: Tom Hanks
year: 
ACTED_IN: The Great Buck Howard, Band of Brothers, Prohibition, Inferno, Ithaca, Bridge of Spies, A Hologram for the King, The Extraordinary Voyage, Everything Is Copy, Sully, The Circle, The Sixties, And the Oscar Goes To..., Small Fry, Hawaiian Vacation, Elvis Has Left the Building, Toy Story That Time Forgot, Partysaurus Rex, Extremely Loud & Incredibly Close, Toy Story of Terror!, The War, The Rutles 2: Can't Buy Me Lunch, Killing Lincoln, Saving Mr. Banks, The Pixar Story, Captain Phillips, Larry Crowne, Cloud Atlas, Toy Story 3, The Da Vinci Code, Who Killed the Electric Car?, The Simpsons Movie, Toy Story, Bachelor Party, You've Got Mail, Apollo 13, Saving Private Ryan, Return with Honor, Dragnet, He Knows You're Alone, Toy Story 2, The Terminal, Sleepless in Seattle, A League of Their Own, Punchline, Nothing in Common, The Green Mile, Turner & Hooch, The 'Burbs, The Ladykillers, Catch Me If You Can, Big, Road to Perdition, Philadelphia, The B

In [ ]:
reset_knowledge_graph()

[LOG] Starting knowledge graph reset...
[LOG] Dropping known full-text indices (if they exist)...
[LOG]   - Attempted to drop index 'movie'.
[LOG]   - Attempted to drop index 'person'.
[2025-04-09T22:50:27.418+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintDoesNotExist} {category: SCHEMA} {title: `DROP INDEX index_87a50586 IF EXISTS` has no effect.} {description: `index_87a50586` does not exist.} {position: None} for query: 'DROP INDEX index_87a50586 IF EXISTS;'
[LOG]   - Attempted to drop index 'index_87a50586'.
[2025-04-09T22:50:27.440+0200] {result.py:332} INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintDoesNotExist} {category: SCHEMA} {title: `DROP INDEX index_45eed07a IF EXISTS` has no effect.} {description: `index_45eed07a` does not exist.} {position: None} for query: 'DROP INDEX index_45eed07a IF EXISTS;'